In [ ]:
# Mengimport library
import nltk
import re
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np

In [ ]:
# Mengunduh resource NLTK yang diperlukan
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
# Lemmatizer dan stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('indonesian'))

In [ ]:
# Membaca kalimat dari file edukasi26.txt
file_path = '/content/drive/MyDrive/Colab Notebooks/NLP/edukasi26.txt'
with open(file_path, 'r') as file:
    corpus = file.read()
    print(corpus)

In [ ]:
# melakukan tokenisasi atau pemisahan atar kalimat
corpus = sent_tokenize(corpus)
corpus

In [ ]:
# Preprocessing function
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Menghapus angka dan karakter khusus
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [ ]:
# Menghapus karakter yang tidak penting pada corpus
cleanCorpus = [preprocess_text(sent) for sent in corpus]
cleanCorpus

In [ ]:
# Tokenize dan bersihkan corpus
tokens = [['<s>']+nltk.word_tokenize(sent)+['</s>'] for sent in cleanCorpus]
tokens

In [ ]:
# Menghitung jumlah token maksimal dari kalimat yang akan diproses
max_tokens = max(len(sentence) for sentence in tokens)

In [ ]:
# Membuat pasangan kata dari 1 hingga max_tokens (jumlah token terbanyak dari kalimat yg akan di proses)
list_ngrams = [Counter() for _ in range(max_tokens)]
for sentence in tokens:
    for i in range(max_tokens):
        list_ngrams[i].update(ngrams(sentence, i + 1))

In [ ]:
# Membuat fungsi interpolasi untuk menghitung nilai probabilitas
def interpolate(checker_words):
    temp = 0.0
    lambda_value = 0.1  # Inisialisasi nilai lambda
    for i in range(len(checker_words)):
        ngram = list_ngrams[i]  # Untuk menghitung jumlah pasangan kata
        sent = checker_words[len(checker_words) - i - 1:len(checker_words)]  # Menentukan kalimat yang ingin dicari probabilitasnya

        if i == 0:
            # Menghitung P_ngram untuk unigram
            P_ngram = ngram[tuple(sent)] / sum(ngram.values()) if tuple(sent) in ngram else 0
        else:
            # Digunakan untuk menghitung jumlah pasangan kata tetapi tidak termasuk kata terakhir
            ngram_before = list_ngrams[i - 1]
            P_ngram = (ngram[tuple(sent)] / ngram_before[tuple(sent[0:len(sent) - 1])]) if tuple(sent) in ngram else 0

        if P_ngram == 0:
            break  # Jika bagian kecil dari pasangan kata saja nol, stop perhitungan

        temp += lambda_value * P_ngram  # Menjumlahkan probabilitas
        lambda_value += 0.3  # Menambah nilai lambda

    return temp  # Mengembalikan nilai probabilitas

In [ ]:
# Cek nilai probabilitas kalimat
kalimat1 = "wisudawan unesa universitas negeri surabaya"
kalimat2 = "mahasiswa memiliki belasan karya yang terindeks sinta dan scopus"
kalimat3 = "garap wisudawan mahasiswa sekolah menengah pertama"

In [ ]:
# Mencetak probabilitas interpolasi untuk setiap kalimat
print(f"Probabilitas '{kalimat1}': {interpolate(preprocess_text(kalimat1).split()):.10f}")

In [ ]:
# Mencetak probabilitas interpolasi untuk setiap kalimat
print(f"Probabilitas '{kalimat2}': {interpolate(preprocess_text(kalimat2).split()):.10f}")

In [ ]:
# Mencetak probabilitas interpolasi untuk setiap kalimat
print(f"Probabilitas '{kalimat3}': {interpolate(preprocess_text(kalimat3).split()):.10f}")